# [Author : TsukiSama9292 - Github Link](https://github.com/tsukisama9292)
## [References - Link](https://ianwu.tw/press/topic/command_line_program/yt-dlp.html#%E4%BB%8B%E7%B4%B9)

# Dependency Installation

In [ ]:
!pip install -q yt-dlp
!pip install -q -U gradio

In [ ]:
### When encountering issues, consider installing it.
# !pip install setuptools

# Using GUI as the download tool. The default is the highest video and audio quality.  

In [ ]:
import gradio as gr
import subprocess
import zipfile
import os
from datetime import datetime

now = datetime.now()
year = now.year
month = now.month
day = now.day
hour = now.hour
minute = now.minute
second = now.second

def download_video(YOUTUBE_URL, do_zip):
    if(do_zip):
        process = subprocess.Popen(
            ['yt-dlp', '-o', 'Download_TMP/%(title)s.%(ext)s', YOUTUBE_URL],
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE, 
            text=True
        )
    else:
        process = subprocess.Popen(
            ['yt-dlp', '-o', 'Download/%(title)s.%(ext)s', YOUTUBE_URL],
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE,  
            text=True 
        )
    for line in process.stdout:
        yield line
    for line in process.stderr:
        yield line
    process.wait()

    if (do_zip):
        zip_files('video')
    
    yield "Video download complete"

def download_audio(YOUTUBE_URL, do_zip):
    if(do_zip):
        process = subprocess.Popen(
            ['yt-dlp', '-o', 'Download_TMP/%(title)s.%(ext)s', '--extract-audio', '--audio-format', 'mp3', YOUTUBE_URL],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
    else:
        process = subprocess.Popen(
            ['yt-dlp', '-o', 'Download/%(title)s.%(ext)s', '--extract-audio', '--audio-format', 'mp3', YOUTUBE_URL],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
    for line in process.stdout:
        yield line

    for line in process.stderr:
        yield line

    process.wait()

    if (do_zip):
        zip_files('audio')

    yield "Audio download complete"

def zip_files(Type):
    zip_folder = 'Download_ZIP'
    if not os.path.exists(zip_folder):
        os.makedirs(zip_folder)
    folder_to_zip = 'Download_TMP'
    zip_filename = f'{zip_folder}/{Type}_{year}_{month}_{day}_{hour}_{minute}_{second}.zip'
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, dirs, files in os.walk(folder_to_zip):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_zip))
    !rm -rf Download_TMP

def gradio_interface(function_choice, YOUTUBE_URL, do_zip):
    if function_choice == "Download video":
        yield from download_video(YOUTUBE_URL, do_zip)
    elif function_choice == "Download audio":
        yield from download_audio(YOUTUBE_URL, do_zip)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Radio(["Download video", "Download audio"], label="Select function"),
        gr.Textbox(label="Enter the link of the YouTube playlist or video"),
        gr.Checkbox(label="Compress into ZIP file")
    ],
    outputs="text",
)
iface.queue().launch(share=True)
# 4K 60fps Video URL (for test) : https://youtu.be/3aRJtJRa434
# Playlists and videos should not use sharing links (ending with 'share'). Please use regular URLs, such as: https://www.youtube.com/playlist?list=PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL